In [52]:
#!/usr/bin/env python
#encoding: utf-8
import sys
# import colored
# import imp
# imp.reload(sys)
# # sys.setdefaultencoding('utf-8')
sys.path.append("../")

from model_trainer.base_task import RNN, Attention_RNN1, Attention_RNN2, Attention_RNN3, Attention_RNN4, \
    Attention_RNN5, Attention_RNN6, CNN
from record import do_record
from confusion_matrix import Alphabet
from confusion_matrix import ConfusionMatrix
import datetime
from tensorflow.contrib import learn
import config
import data_helpers
import util
import tensorflow as tf
import numpy as np
from scorer import get_rank_score_by_file
import time

from pycorenlp import StanfordCoreNLP
import re
from tree_util import ConstTree, DepTree, ConstNode, DepNode
import tree_util
from vocab import Vocab
import random
import tf_tree_task

PAD_WORD = '<blank>'
UNK_WORD = '<unk>'
BOS_WORD = '<s>'
EOS_WORD = '</s>'
NUM_WORD = '<num>'

PUNC_TAG = '<punc>'

import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'

# cd D:\stanford-corenlp-full-2016-10-31
# java -mx4g -cp "*" edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000 -timeout 15000


train_data_dir = 'D:/PY/Pycode/project/end-to-end-discourse-parser/data/gen_my_four_sen/imp'   # 'D:/PY/Pycode/project/end-to-end-discourse-parser/data/four_way/PDTB_imp'
# 准备原始数据文本
train_arg1s, train_arg2s, train_labels = data_helpers.load_data_and_labels("%s/train" % train_data_dir)
dev_arg1s, dev_arg2s, dev_labels = data_helpers.load_data_and_labels("%s/dev" % train_data_dir)
test_arg1s, test_arg2s, test_labels = data_helpers.load_data_and_labels("%s/test" % train_data_dir)

num_classes = train_labels.shape[1]


print("num_classes", num_classes)
print('train/dev/test:',len(train_arg1s),len(dev_arg1s),len(test_arg1s))

core_nlp = StanfordCoreNLP('http://localhost:9000')
annotate_func = lambda x: core_nlp.annotate(x, properties={
    'annotators': 'tokenize,ssplit,pos,lemma,parse,depparse',
    'outputFormat': 'json',
    # 'ssplit.isOneSentence': True
})

def extract_json_ob(json_ob, lowercase=True, use_lemma=True, replace_num=True):
    processed_words = []
    dep_trees = []
    const_trees = []
    # there may be multiple sentences
    for sentence_info in json_ob['sentences']:
        # extract the words of the sentence
        sent_words = []
        for token in sentence_info['tokens']:
            word = token['lemma'] if use_lemma else token['word']
            word = word.lower() if lowercase else word
            if replace_num and any(c.isdigit() for c in word):
                word = re.sub('[.|,|/| ]', '', word.lstrip('-'))
                if word.isdigit():
                    word = NUM_WORD
            sent_words.append(word)
        # build the dependency tree
        nodes = [DepNode(len(processed_words) + word_idx) for word_idx in range(len(sent_words))]
        root_node = None
        for dep in sentence_info['basicDependencies']:
            if dep['governor'] == 0:
                root_node = nodes[dep['dependent'] - 1]
            else:
                nodes[dep['governor'] - 1].add_child(nodes[dep['dependent'] - 1])
        dep_tree = DepTree()
        dep_tree.assign_root(root_node)
        dep_trees.append(dep_tree)
        # build the constituency tree
        const_tree = ConstTree()
        const_tree.load_from_string(sentence_info['parse'])
        const_tree.compress()
        const_tree.binarize()
        const_trees.append(const_tree)
        # add the sent_words to processed_words
        processed_words.extend(sent_words)
    if len(dep_trees) > 1:
        for dep_tree in dep_trees[1:]:
            dep_trees[0].merge(dep_tree)
    if len(const_trees) > 1:
        for const_tree in const_trees[1:]:
            const_trees[0].merge(const_tree)
        const_trees[0].binarize()
    return processed_words, dep_trees[0], const_trees[0]


class PDTBInstance:
    def __init__(self):
        self.left_words = []
        self.right_words = []
        self.left_const_tree = None
        self.right_const_tree = None
        self.left_dep_tree = None
        self.right_dep_tree = None
        self.label = None

# train_arg1s_parse_result = []
def data2Inst(arg1s, arg2s, label):
    data_set = []
    for arg1,arg2, lab in zip(arg1s,arg2s,label):
        inst = PDTBInstance()
        arg1_parse_result = annotate_func(arg1)
        arg2_parse_result = annotate_func(arg2)
        # train_arg1s_parse_result.append(parse_result)
        inst.left_words, inst.left_dep_tree, inst.left_const_tree = extract_json_ob(arg1_parse_result)
        inst.right_words, inst.right_dep_tree, inst.right_const_tree = extract_json_ob(arg2_parse_result)
        inst.label = lab
        data_set.append(inst)
    return data_set

# {list}, every element is a {PDTBInstance}
# train_set = data2Inst(train_arg1s, train_arg2s, train_labels)
dev_set = data2Inst(dev_arg1s, dev_arg2s, dev_labels)################# test
# test_set = data2Inst(test_arg1s, test_arg2s, test_labels)

# PDTBDataSet function, get all words in dataset
def get_all_words(dataset):
    words = []
    for inst in dataset:
        words.extend(inst.left_words)
        words.extend(inst.right_words)
    return words

def get_all_tags(dataset):
    tags = []
    for inst in dataset:
        tags.extend([node.tag for node in inst.left_const_tree.bfs_tranverse()])
        tags.extend([node.tag for node in inst.right_const_tree.bfs_tranverse()])
    return tags


### load embedding ###
word_vocab = Vocab(mannual_add=[PAD_WORD, UNK_WORD, BOS_WORD, EOS_WORD, NUM_WORD])
# for word in get_all_words(train_set)+get_all_words(dev_set)+get_all_words(test_set):
for word in get_all_words(dev_set): ################# test
    word_vocab.add(word)

# word_vocab.load_pretrained_emb(PathConfig.embedding_path)
embedding_path='D:\\data\\glove.6B\\glove.6B.50d.txt'
word_vocab.load_pretrained_emb(embedding_path)  # save as 'word_vocab.obj'


# load tag embedding
tag_vocab = Vocab()
# for tag in get_all_tags(train_set)+get_all_tags(dev_set)+get_all_tags(test_set):################# test
for tag in get_all_tags(dev_set):
    tag_vocab.add(tag)
print('Size of tag vocab: {}'.format(tag_vocab.size()))
# tag_vocab.init_embed(ModelConfig.tag_embed_dim)
tag_embed_dim = 50
tag_vocab.init_embed(tag_embed_dim) # save as 'tag_vocab.obj'
# torch.save(tag_vocab, os.path.join(PathConfig.experiment_data_dir, 'tag_vocab.obj'))

# dev_set.format_instances_to_torch_var(word_vocab, tag_vocab) 将树节点的单词、tag等换为torch的数字 save as dataset

print('mmm')
print('fff')
### load embedding end  ###

def transword2id(data):
    for i,tdata in enumerate(data):
        for j,word in enumerate(tdata.left_words):
            # print(word)
            if word in word_vocab.token2id:
                # print(word_vocab.token2id[word])
                data[i].left_words[j]=word_vocab.token2id[word]
            else:
                data[i].left_words[j]=0
        for j,word in enumerate(tdata.right_words):
            # print(word)
            if word in word_vocab.token2id:
                # print(word_vocab.token2id[word])
                data[i].right_words[j]=word_vocab.token2id[word]
            else:
                data[i].right_words[j]=0
transword2id(dev_set)

class TreeConfig(object):

    num_emb=None

    emb_dim = 300
    hidden_dim = 150
    output_dim=None
    degree = 2

    num_epochs = 10
    early_stopping = 2
    dropout = 0.5
    lr = 0.05
    emb_lr = 0.1
    reg=0.0001

    batch_size = 5
    #num_steps = 10
    maxseqlen = None
    maxnodesize = None
    fine_grained=False
    trainable_embeddings=True
    nonroot_labels=True
    #dependency=True not supported

treeconfig=TreeConfig()
num_emb = len(word_vocab.id2token)

treeconfig.num_emb = num_emb
# config.output_dim = num_labels
treeconfig.output_dim = num_classes

treeconfig.maxseqlen = tree_util.get_max_len_data(dev_set) # utils.get_max_len_data(data)
treeconfig.maxnodesize = tree_util.get_max_node_size(dev_set) #utils.get_max_node_size(data)
print((treeconfig.maxnodesize,treeconfig.maxseqlen ," maxsize"))

random.seed()
np.random.seed()

num_classes 4
train/dev/test: 13680 541 799
Size of tag vocab: 64
mmm
fff
(205, 101, ' maxsize')


In [53]:
import numpy as np
import tensorflow as tf
# import os
import sys

# from tf_data_utils import extract_tree_data, extract_batch_tree_data
# from tf_data_utils import extract_batch_tree_data


class tf_NarytreeLSTM(object):

    def __init__(self, config):
        self.emb_dim = config.emb_dim
        self.hidden_dim = config.hidden_dim
        self.num_emb = config.num_emb
        self.output_dim = config.output_dim
        self.config = config
        self.batch_size = config.batch_size
        self.reg = self.config.reg
        self.degree = config.degree
        assert self.emb_dim > 1 and self.hidden_dim > 1

        self.add_placeholders()

        emb_leaves = self.add_embedding()

        self.add_model_variables()

        batch_loss = self.compute_loss(emb_leaves)

        self.loss, self.total_loss = self.calc_batch_loss(batch_loss)

        self.train_op1, self.train_op2 = self.add_training_op()

    def add_embedding(self):

        # embed=np.load('glove{0}_uniform.npy'.format(self.emb_dim))
        with tf.variable_scope("Embed", regularizer=None):
            embedding = tf.get_variable('embedding', [self.num_emb,
                                                      self.emb_dim]
                                        , initializer=tf.random_uniform_initializer(-0.05, 0.05), trainable=True,
                                        regularizer=None)
            ix = tf.to_int32(tf.not_equal(self.input, -1)) * self.input
            emb_tree = tf.nn.embedding_lookup(embedding, ix)
            emb_tree = emb_tree * (tf.expand_dims(
                tf.to_float(tf.not_equal(self.input, -1)), 2))

            return emb_tree

    def add_placeholders(self):
        dim2 = self.config.maxnodesize
        dim1 = self.config.batch_size
        self.input = tf.placeholder(tf.int32, [dim1, dim2], name='input')
        self.treestr = tf.placeholder(tf.int32, [dim1, dim2, 2], name='tree')
        self.labels = tf.placeholder(tf.int32, [dim1, dim2], name='labels')
        self.dropout = tf.placeholder(tf.float32, name='dropout')

        self.n_inodes = tf.reduce_sum(tf.to_int32(tf.not_equal(self.treestr, -1)), [1, 2])
        self.n_inodes = self.n_inodes / 2

        self.num_leaves = tf.reduce_sum(tf.to_int32(tf.not_equal(self.input, -1)), [1])
        self.batch_len = tf.placeholder(tf.int32, name="batch_len")

    def calc_wt_init(self, fan_in=300):
        eps = 1.0 / np.sqrt(fan_in)
        return eps

    def add_model_variables(self):

        with tf.variable_scope("Composition",
                               initializer=
                               tf.contrib.layers.xavier_initializer(),
                               regularizer=
                               tf.contrib.layers.l2_regularizer(self.config.reg
                                                                )):
            cU = tf.get_variable("cU", [self.emb_dim, 2 * self.hidden_dim],
                                 initializer=tf.random_uniform_initializer(-self.calc_wt_init(), self.calc_wt_init()))
            cW = tf.get_variable("cW", [self.degree * self.hidden_dim, (self.degree + 3) * self.hidden_dim],
                                 initializer=tf.random_uniform_initializer(-self.calc_wt_init(self.hidden_dim),
                                                                           self.calc_wt_init(self.hidden_dim)))
            cb = tf.get_variable("cb", [4 * self.hidden_dim], initializer=tf.constant_initializer(0.0),
                                 regularizer=tf.contrib.layers.l2_regularizer(0.0))
            # cU = tf.get_variable("cU",[self.emb_dim,2*self.hidden_dim])
            # cW = tf.get_variable("cW",[self.degree*self.hidden_dim,(self.degree+3)*self.hidden_dim])
            # cb = tf.get_variable("cb",[4*self.hidden_dim],initializer=tf.constant_initializer(0.0),regularizer=tf.contrib.layers.l2_regularizer(0.0))
        with tf.variable_scope("Projection", regularizer=tf.contrib.layers.l2_regularizer(self.config.reg)):
            U = tf.get_variable("U", [self.output_dim, self.hidden_dim],
                                initializer=tf.random_uniform_initializer(self.calc_wt_init(self.hidden_dim),
                                                                          self.calc_wt_init(self.hidden_dim))
                                )
            bu = tf.get_variable("bu", [self.output_dim], initializer=
            tf.constant_initializer(0.0), regularizer=tf.contrib.layers.l2_regularizer(0.0))

    def process_leafs(self, emb):

        with tf.variable_scope("Composition", reuse=True):
            cU = tf.get_variable("cU", [self.emb_dim, 2 * self.hidden_dim])
            cb = tf.get_variable("cb", [4 * self.hidden_dim])
            b = tf.slice(cb, [0], [2 * self.hidden_dim])

            def _recurseleaf(x): # x:(300,)
                concat_uo = tf.matmul(tf.expand_dims(x, 0), cU) + b # cU:(300,300) b:(300,)---> concat_uo(1,300)
                # u, o = tf.split(1, 2, concat_uo)
                u, o = tf.split(concat_uo, 2, 1)
                o = tf.nn.sigmoid(o)
                u = tf.nn.tanh(u)

                c = u  # tf.squeeze(u)
                h = o * tf.nn.tanh(c)

                hc = tf.concat([h, c], 1)
                hc = tf.squeeze(hc)
                return hc

        hc = tf.map_fn(_recurseleaf, emb) # emb:(?,300)
        return hc

    def compute_loss(self, emb_batch, curr_batch_size=None):
        outloss = []
        prediction = []
        for idx_batch in range(self.config.batch_size):
            tree_states = self.compute_states(emb_batch, idx_batch)
            logits = self.create_output(tree_states)

            labels1 = tf.gather(self.labels, idx_batch)
            labels2 = tf.reduce_sum(tf.to_int32(tf.not_equal(labels1, -1)))
            labels = tf.gather(labels1, tf.range(labels2))
            loss = self.calc_loss(logits, labels)

            pred = tf.nn.softmax(logits)

            pred_root = tf.gather(pred, labels2 - 1)

            prediction.append(pred_root)
            outloss.append(loss)

        batch_loss = tf.stack(outloss)
        self.pred = tf.stack(prediction)

        return batch_loss

    def compute_states(self, emb, idx_batch=0):

        num_leaves = tf.squeeze(tf.gather(self.num_leaves, idx_batch))
        # num_leaves=tf.Print(num_leaves,[num_leaves])
        n_inodes = tf.gather(self.n_inodes, idx_batch)
        # embx=tf.gather(emb,tf.range(num_leaves))
        embx = tf.gather(tf.gather(emb, idx_batch), tf.range(num_leaves))
        # treestr=self.treestr#tf.gather(self.treestr,tf.range(self.n_inodes))
        treestr = tf.gather(tf.gather(self.treestr, idx_batch), tf.cast(tf.range(n_inodes), dtype=tf.int32))
        leaf_hc = self.process_leafs(embx)
        leaf_h, leaf_c = tf.split(leaf_hc, 2, 1)

        node_h = tf.identity(leaf_h)
        node_c = tf.identity(leaf_c)

        idx_var = tf.constant(0)  # tf.Variable(0,trainable=False)

        with tf.variable_scope("Composition", reuse=True):
            cW = tf.get_variable("cW", [self.degree * self.hidden_dim, (self.degree + 3) * self.hidden_dim])
            cb = tf.get_variable("cb", [4 * self.hidden_dim])
            bu, bo, bi, bf = tf.split(cb, 4, 0)

            def _recurrence(node_h, node_c, idx_var):
                node_info = tf.gather(treestr, idx_var)

                child_h = tf.gather(node_h, node_info)
                child_c = tf.gather(node_c, node_info)

                flat_ = tf.reshape(child_h, [-1])
                tmp = tf.matmul(tf.expand_dims(flat_, 0), cW)
                u, o, i, fl, fr = tf.split(tmp, 5, 1)

                i = tf.nn.sigmoid(i + bi)
                o = tf.nn.sigmoid(o + bo)
                u = tf.nn.tanh(u + bu)
                fl = tf.nn.sigmoid(fl + bf)
                fr = tf.nn.sigmoid(fr + bf)

                f = tf.concat([fl, fr], 0)
                c = i * u + tf.reduce_sum(f * child_c, [0])
                h = o * tf.nn.tanh(c)

                node_h = tf.concat([node_h, h], 0)

                node_c = tf.concat([node_c, c], 0)

                idx_var = tf.add(idx_var, 1)

                return node_h, node_c, idx_var

            # loop_cond = lambda a1, b1, idx_var: tf.less(idx_var, n_inodes)
            loop_cond = lambda a1, b1, idx_var: tf.less(tf.cast(idx_var, tf.float64), n_inodes)


            loop_vars = [node_h, node_c, idx_var]
            node_h, node_c, idx_var = tf.while_loop(loop_cond, _recurrence,
                                                    loop_vars, parallel_iterations=10)

            return node_h

    def create_output(self, tree_states):

        with tf.variable_scope("Projection", reuse=True):
            U = tf.get_variable("U", [self.output_dim, self.hidden_dim],
                                )
            bu = tf.get_variable("bu", [self.output_dim])

            h = tf.matmul(tree_states, U, transpose_b=True) + bu
            return h

    def calc_loss(self, logits, labels):

        l1 = tf.nn.sparse_softmax_cross_entropy_with_logits(
             labels=labels, logits=logits)
        loss = tf.reduce_sum(l1, [0])
        return loss

    def calc_batch_loss(self, batch_loss):
        reg_losses = tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)
        regpart = tf.add_n(reg_losses)
        loss = tf.reduce_mean(batch_loss)
        total_loss = loss + 0.5 * regpart
        return loss, total_loss

    def add_training_op_old(self):

        opt = tf.train.AdagradOptimizer(self.config.lr)
        train_op = opt.minimize(self.total_loss)
        return train_op

    def add_training_op(self):
        loss = self.total_loss
        opt1 = tf.train.AdagradOptimizer(self.config.lr)
        opt2 = tf.train.AdagradOptimizer(self.config.emb_lr)

        ts = tf.trainable_variables()
        gs = tf.gradients(loss, ts)
        gs_ts = list(zip(gs, ts))

        gt_emb, gt_nn = [], []
        for g, t in gs_ts:
            # print t.name,g.name
            if "Embed/embedding:0" in t.name:
                # g=tf.Print(g,[g.get_shape(),t.get_shape()])
                gt_emb.append((g, t))
                # print t.name
            else:
                gt_nn.append((g, t))
                # print t.name

        train_op1 = opt1.apply_gradients(gt_nn)
        train_op2 = opt2.apply_gradients(gt_emb)
        train_op = [train_op1, train_op2]

        return train_op

    def train(self, data, sess):
        from random import shuffle
        data_idxs = list(range(len(data)))
        # shuffle(data_idxs) ################################test
        losses = []
        for i in range(0, len(data), self.batch_size):
            batch_size = min(i + self.batch_size, len(data)) - i
            if batch_size < self.batch_size: break

            batch_idxs = data_idxs[i:i + batch_size]
            batch_data = [data[ix] for ix in batch_idxs]  # [i:i+batch_size]

            input_b = batch_data
            input_b, treestr_b, labels_b = extract_batch_tree_data(batch_data, self.config.maxnodesize)

            feed = {self.input: input_b, self.treestr: treestr_b, self.labels: labels_b,
                    self.dropout: self.config.dropout, self.batch_len: len(input_b)}

            loss, _, _ = sess.run([self.loss, self.train_op1, self.train_op2], feed_dict=feed)
            # sess.run(self.train_op,feed_dict=feed)

            losses.append(loss)
            avg_loss = np.mean(losses)
            sstr = 'avg loss %.2f at example %d of %d\r' % (avg_loss, i, len(data))
            sys.stdout.write(sstr)
            sys.stdout.flush()

            # if i>1000: break
        return np.mean(losses)

    def evaluate(self, data, sess):
        num_correct = 0
        total_data = 0
        data_idxs = list(range(len(data)))
        test_batch_size = self.config.batch_size
        losses = []
        for i in range(0, len(data), test_batch_size):
            batch_size = min(i + test_batch_size, len(data)) - i
            if batch_size < test_batch_size: break
            batch_idxs = data_idxs[i:i + batch_size]
            batch_data = [data[ix] for ix in batch_idxs]  # [i:i+batch_size]
            labels_root = [l for _, l in batch_data]
            input_b, treestr_b, labels_b = extract_batch_tree_data(batch_data, self.config.maxnodesize)

            feed = {self.input: input_b, self.treestr: treestr_b, self.labels: labels_b, self.dropout: 1.0,
                    self.batch_len: len(input_b)}

            pred_y = sess.run(self.pred, feed_dict=feed)
            # print pred_y,labels_root
            y = np.argmax(pred_y, axis=1)
            # num_correct+=np.sum(y==np.array(labels_root))
            for i, v in enumerate(labels_root):
                if y[i] == v: num_correct += 1
                total_data += 1
            # break

        acc = float(num_correct) / float(total_data)
        return acc

In [9]:
with tf.Graph().as_default():
    model = tf_tree_task.tf_NarytreeLSTM(treeconfig)

    init = tf.initialize_all_variables()
    saver = tf.train.Saver()
    best_valid_score = 0.0
    best_valid_epoch = 0
    with tf.Session() as sess:
        sess.run(init)
        start_time = time.time()

        # if restore: saver.restore(sess, './ckpt/tree_rnn_weights')
        for epoch in range(treeconfig.num_epochs):
            print(('epoch', epoch))
            avg_loss = 0.0
            # avg_loss = train_epoch(model, train_set, sess)
            avg_loss = model.train(dev_set,sess)
            print(('avg loss', avg_loss))

INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.


c:\python35\lib\site-packages\tensorflow\python\ops\gradients_impl.py:91: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Instructions for updating:
Use `tf.global_variables_initializer` instead.
('epoch', 0)


NameError: name 'extract_batch_tree_data' is not defined

In [8]:
print(treeconfig)

In [56]:
import copy
data = copy.deepcopy(dev_set)

In [18]:
data_idxs = list(range(len(data)))
losses=[]
# for i in range(0, len(data), 5):
for i in range(0,20,5):
    batch_size = min(i + 5, len(data)) - i
    if batch_size < 5: break

    batch_idxs = data_idxs[i:i + batch_size]
    batch_data = [data[ix] for ix in batch_idxs]
    print(batch_data)
    print('--'*30)
    

[<__main__.PDTBInstance object at 0x000000220C079EF0>, <__main__.PDTBInstance object at 0x000000220C5A0048>, <__main__.PDTBInstance object at 0x000000220C5A0D68>, <__main__.PDTBInstance object at 0x000000220C57CF28>, <__main__.PDTBInstance object at 0x000000220C5AAEB8>]
------------------------------------------------------------
[<__main__.PDTBInstance object at 0x000000220C5AE8D0>, <__main__.PDTBInstance object at 0x000000220C598E10>, <__main__.PDTBInstance object at 0x000000220C5C0E10>, <__main__.PDTBInstance object at 0x000000220C5C0BA8>, <__main__.PDTBInstance object at 0x000000220C5D2EF0>]
------------------------------------------------------------
[<__main__.PDTBInstance object at 0x000000220C5D87F0>, <__main__.PDTBInstance object at 0x000000220C5F3C18>, <__main__.PDTBInstance object at 0x000000220C5FE198>, <__main__.PDTBInstance object at 0x000000220C606DD8>, <__main__.PDTBInstance object at 0x000000220C606198>]
------------------------------------------------------------
[<__

In [57]:
tdata = data[0:5]

In [58]:
for da in data:
    print(da.left_words)

[531, 2653, 2324, 2396, 1373, 1892, 2884, 2139, 1261, 0, 257, 2324, 527, 0, 2324, 1367, 3026, 531, 692, 1288, 122, 687, 2627, 649, 2702, 1718, 1327, 2813, 1279, 0, 681, 1593, 959, 1794, 2787, 280, 1367, 1315, 543, 2108, 1974, 531, 2987, 524, 2794]
[1523, 531, 633, 1379, 2384, 1823, 2529, 938, 1718, 2873, 820, 531, 3102, 2660, 2517, 959, 280, 543, 186, 2324, 2539, 531, 2660, 2442, 543, 186, 2635, 2794]
[1823, 1886, 543, 1279, 1831, 280, 3077, 2813, 2873, 543, 1726, 2324, 938, 2777, 1524, 3060, 2324, 491, 2817, 2517, 531, 633, 2794]
[434, 1163, 2529, 1718, 2324, 1535, 2777, 2324, 531, 633, 1367, 543, 2740, 2873, 818, 1163, 959, 692, 2539, 2667, 2796, 543, 2627, 1327, 186, 531, 3102, 2660, 2794]
[1823, 280, 2460, 2079, 2873, 1279, 1414, 1810, 531, 1017, 687, 183, 2906, 1461, 2324, 1318, 2465, 2873, 1215, 1654, 2517, 1212, 2324, 938, 1535, 2777, 2794]
[1041, 1974, 531, 482, 2517, 217, 666, 0, 1919, 782, 2906, 531, 169, 280, 543, 368, 2324, 1279, 2584, 1590, 652, 2798, 543, 77, 581, 2560, 1

In [46]:
def transword2id(data):
    for i,tdata in enumerate(data):
        for j,word in enumerate(tdata.left_words):
            print(word)
            if word in word_vocab.token2id:
                print(word_vocab.token2id[word])
                data[i].left_words[j]=word_vocab.token2id[word]
            else:
                data[i].left_words[j]=0

In [49]:
transword2id(data)

531
2653
2324
2396
1373
1892
2884
2139
1261
0
257
2324
527
0
2324
1367
3026
531
692
1288
122
687
2627
649
2702
1718
1327
2813
1279
0
681
1593
959
1794
2787
280
1367
1315
543
2108
1974
531
2987
524
2794
1523
531
633
1379
2384
1823
2529
938
1718
2873
820
531
3102
2660
2517
959
280
543
186
2324
2539
531
2660
2442
543
186
2635
2794
1823
1886
543
1279
1831
280
3077
2813
2873
543
1726
2324
938
2777
1524
3060
2324
491
2817
2517
531
633
2794
434
1163
2529
1718
2324
1535
2777
2324
531
633
1367
543
2740
2873
818
1163
959
692
2539
2667
2796
543
2627
1327
186
531
3102
2660
2794
1823
280
2460
2079
2873
1279
1414
1810
531
1017
687
183
2906
1461
2324
1318
2465
2873
1215
1654
2517
1212
2324
938
1535
2777
2794
hard
1041
on
1974
the
531
heel
482
of
2517
friday
217
's
666
190-point
stock-market
1919
plunge
782
and
2906
the
531
uncertainty
169
that
280
be
543
follow
368
,
2324
a
1279
few
2584
big
1590
brokerage
652
firm
2798
be
543
roll
77
out
581
new
2560
ad
142
trumpet
2152
a
1279
familiar
422
message
7

``
1823
when
2094
it
1886
go
1056
,
2324
you
2801
never
65
know
215
when
2094
you
2801
will
1709
get
95
it
1886
back
956
,
2324
''
938
he
1535
say
2777
.
2794
but
1523
per-share
profit
1333
drop
2896
only
707
<num>
4
%
2743
to
2873
$
2382
<num>
4
a
1279
share
365
from
687
$
2382
<num>
4
a
1279
share
365
,
2324
as
1762
the
531
company
419
continue
1748
its
2096
stock
258
buy-back
2078
plan
1542
.
2794
average
2221
share
365
outstanding
54
drop
2896
to
2873
<num>
4
million
448
from
687
<num>
4
million
448
.
2794
mr.
2175
hingorani
413
say
2777
he
1535
lower
533
he
1535
full-year
38
estimate
1235
for
2593
<num>
4
to
2873
$
2382
<num>
4
a
1279
share
365
from
687
$
2382
<num>
4
a
1279
share
365
.
2794
mr.
2175
hingorani
413
say
2777
he
1535
lower
533
he
1535
full-year
38
estimate
1235
for
2593
<num>
4
to
2873
$
2382
<num>
4
a
1279
share
365
from
687
$
2382
<num>
4
a
1279
share
365
.
2794
revenue
1655
projection
1622
be
543
slash
2977
to
2873
$
2382
<num>
4
billion
2380
from
687
$
2382
<num>

its
2096
tender
1941
offer
2350
,
2324
which
1318
challenge
185
a
1279
agreement
2703
between
855
bellsouth
2404
corp.
122
and
2906
lin
2272
to
2873
merge
2841
they
2228
cellular-telephone
business
2395
.
2794
the
531
lin
2272
board
1401
say
2777
holder
763
may
345
be
543
mislead
1554
by
1327
the
531
provision
1353
in
971
the
531
mccaw
2483
proposal
1414
that
280
``
1823
guarantee
332
''
938
private
1953
market
3056
value
134
after
2354
five
1320
year
2111
for
2593
the
531
remain
644
share
365
.
2794
mccaw
2483
have
2813
``
1823
no
3064
obligation
1505
to
2873
purchase
1678
and
2906
the
531
definition
2319
of
2517
private
1953
market
3056
value
134
be
543
uncertain
510
,
2324
''
938
the
531
lin
2272
board
1401
say
2777
.
2794
dow
2705
jones
2783
,
2324
publisher
1111
of
2517
the
531
wall
867
street
1040
journal
2361
,
2324
have
2813
launch
2163
a
1279
$
2382
18-a-share
,
2324
or
1593
$
2382
<num>
4
million
448
,
2324
tender
1941
offer
2350
to
2873
acquire
2697
the
531
remain
644
telera

have
2813
good
2260
education
149
,
2324
the
531
national
1270
association
764
of
2517
temporary
649
services
942
say
2777
;
393
its
2096
survey
1798
of
2517
<num>
4
such
2069
employee
1582
show
2243
<num>
4
%
2743
with
1531
more
1747
than
974
a
1279
high-school
875
education
149
,
2324
and
2906
<num>
4
%
2743
with
1531
college
379
degree
2209
.
2794
about
2671
<num>
4
%
2743
reimburse
2666
for
2593
all
320
or
1593
some
1750
loss
2590
.
2794
since
2949
<num>
4
,
2324
more
1747
company
419
give
585
sales-loss
aid
1704
,
2324
as
1762
many
537
real-estate
1239
value
134
depreciate
1519
,
2324
the
531
council
334
say
2777
.
2794
rjr
447
nabisco
1180
pay
2910
up
3023
to
2873
$
2382
<num>
4
of
2517
loss
2590
,
2324
include
1892
improvement
2868
.
2794
goodrich
1140
will
1709
not
1315
ensure
2921
loss
2590
coverage
2371
,
2324
but
1523
will
1709
prevent
3026
a
1279
``
1823
catastrophic
2253
loss
2590
goodrich
1140
will
1709
not
1315
ensure
2921
loss
2590
coverage
2371
,
2324
but
1523
will
170

3097
-lrb-
put
2586
aside
2946
the
531
artificial
1574
boost
3057
that
280
the
531
tax
1926
code
3080
give
585
to
2873
debt
2787
over
1901
equity
1398
-rrb-
,
2324
and
2906
what
32
we
2353
have
2813
see
738
be
543
the
531
market
3056
rein
2249
they
2228
in
971
.
2794
if
3074
it
1886
be
543
correct
1155
to
2873
find
2839
that
280
concern
2190
over
1901
corporate
337
debt
2787
and
2906
lbo
645
cause
1406
gray
1906
friday
217
,
2324
what
32
be
543
the
531
implication
798
for
2593
policy
2728
maker
1504
?
3002
after
2354
all
320
,
2324
the
531
stock
258
market
3056
's
666
response
1588
to
2873
the
531
collapse
106
of
2517
the
531
ual
2899
deal
2368
might
2883
be
543
take
264
to
2873
confirm
1628
the
531
anti-debt
direction
2360
of
2517
regulator
1806
.
2794
to
2873
the
531
extent
196
that
280
friday
217
's
666
sell-off
876
reflect
1675
a
1279
sudden
1725
reappraisal
2412
of
2517
the
531
excess
3097
of
2517
leverage
1842
,
2324
the
531
message
718
be
543
that
280
wall
867
street
1040
and
29

m.
1532
machold
960
,
2324
director
2817
of
2517
the
531
new
2560
jersey
535
division
578
of
2517
investment
651
,
2324
which
1318
oversee
890
$
2382
<num>
4
billion
2380
in
971
investment
651
,
2324
say
2777
the
531
``
1823
first
1815
thing
2788
we
2353
do
1146
be
543
to
2873
double
2475
we
2353
order
262
''
938
yesterday
939
morning
1119
.
2794
trading
127
in
971
walt
1466
disney
352
co.
2313
particularly
2075
catch
2059
trader
27
'
2574
eye
1305
.
2794
accord
86
to
2873
big
1590
board
1401
official
2320
,
2324
disney
352
have
2813
one
1017
of
2517
the
531
biggest
385
sell-order
imbalance
1207
on
1974
friday
217
around
1793
wall
867
street
1040
,
2324
trading
127
desk
3091
be
543
relieve
1759
that
280
they
2228
could
2796
at
1495
least
1777
play
84
the
531
market
3056
yesterday
939
,
2324
in
971
contrast
2156
to
2873
friday
217
's
666
gridlock
2698
.
2794
at
1495
donaldson
941
,
2324
lufkin
2099
&
592
jenrette
284
inc.
2982
,
2324
head
1968
equity
1398
trader
27
dudley
2553
eppel
say

1279
share
365
.
2794
dig
254
acquisition
2536
corp.
122
,
2324
the
531
new
2560
jersey
535
investor
1510
's
666
acquisition
2536
vehicle
2055
,
2324
say
2777
that
280
as
1762
of
2517
the
531
close
412
of
2517
business
2395
yesterday
939
,
2324
<num>
4
share
365
have
2813
be
543
tender
1941
.
2794
if
3074
you
2801
guess
1711
black-and-white
35
snapshot
1703
,
2324
you
2801
be
543
right
2487
.
2794
trendy
3012
magazine
752
advertisement
824
feature
3035
stark
6
black-and-white
35
photo
1382
of
2517
hollywood
2089
celebrity
2218
pitch
839
jeans
1015
,
2324
shoe
2676
and
2906
liquor
2692
.
2794
what
32
be
543
happen
1641
in
971
photography
2015
mirror
815
the
531
popularity
520
of
2517
black
1224
and
2906
white
202
in
971
fashion
864
,
2324
home
251
furnishings
2406
and
2906
cinematography
2463
.
2794
photographic
2829
company
419
be
543
scramble
3096
to
2873
tap
804
the
531
resurge
1214
market
3056
,
2324
revive
1843
some
1750
black-and-white
35
product
2869
line
2726
and
2906
develop
58

to
2873
redress
1840
that
280
record
1994
in
971
namibia
499
.
2794
copper
1227
and
2906
crude
16
oil
138
react
715
sharply
2791
to
2873
the
531
concern
2190
that
280
a
1279
crash
617
yesterday
939
could
2796
have
2813
a
1279
potentially
2543
devastating
2969
effect
1340
on
1974
the
531
economy
1314
.
2794
copper
1227
fall
917
and
2906
show
2243
little
1664
rebound
2587
through
1131
the
531
day
2461
as
1762
one
1017
of
2517
the
531
major
1688
supply
90
problem
1831
that
280
have
2813
be
543
support
1032
price
1319
appear
2082
to
2873
be
543
solve
1945
.
2794
in
971
commodity
2211
market
3056
yesterday
939
:
2114
precious
223
metals
1483
:
2114
future
1722
price
1319
be
543
moderately
760
higher
1908
as
1762
gold
1912
give
585
up
3023
some
1750
of
2517
its
2096
early
2856
gain
2649
and
2906
platinum
2576
behave
699
independently
693
,
2324
first
1815
fall
917
and
2906
then
884
later
1681
rise
2144
.
2794
the
531
spot
570
october
2031
gold
1912
price
1319
rise
2144
$
2382
<num>
4
to
2873

<num>
4
%
2743
in
971
surprisingly
1116
lackluster
2595
trading
127
.
2794
investor
1510
big
1590
and
2906
small
2397
say
2777
they
2228
learn
987
valuable
2499
lesson
418
since
2949
the
531
<num>
4
crash
617
investor
1510
big
1590
and
2906
small
2397
say
2777
they
2228
learn
987
valuable
2499
lesson
418
since
2949
the
531
<num>
4
crash
617
:
2114
in
971
this
2650
age
746
of
2517
computerized
1682
trading
127
,
2324
huge
1870
correction
1357
or
1593
runup
2646
in
971
a
1279
few
2584
hour
2464
'
2574
time
979
must
932
be
543
expect
2863
.
2794
stephen
3117
boesel
30
,
2324
a
1279
major
1688
money
2878
manager
1643
for
2593
t.
1972
rowe
1565
price
1319
in
971
baltimore
2656
,
2324
say
2777
,
2324
``
1823
there
1049
be
543
less
2258
panic
228
than
974
in
971
<num>
4
in
971
somerset
1951
,
2324
wis.
2121
,
2324
adrian
74
sween
1800
,
2324
who
336
own
2855
a
1279
supplier
1437
of
2517
nursing-home
equipment
2620
and
2906
be
543
not
1315
active
1714
in
971
the
531
stock
258
market
3056
,
232

take
264
the
531
company
419
public
1473
in
971
a
1279
offering
2193
that
280
net
395
applied
545
about
2671
$
2382
<num>
4
million
448
,
2324
which
1318
help
2957
launch
2163
the
531
company
419
's
666
acquisition
2536
program
1252
.
2794
sale
1294
climb
2322
to
2873
a
1279
estimate
1235
$
2382
<num>
4
million
448
in
971
fiscal
1254
<num>
4
,
2324
end
388
aug.
1716
<num>
4
,
2324
from
687
$
2382
<num>
4
million
448
in
971
fiscal
1254
<num>
4
.
2794


In [37]:
get_all_words(tdata)

['the',
 'bill',
 ',',
 'whose',
 'backer',
 'include',
 'chairman',
 'dan',
 'rostenkowski',
 '-lrb-',
 'd.',
 ',',
 'ill.',
 '-rrb-',
 ',',
 'would',
 'prevent',
 'the',
 'resolution',
 'trust',
 'corp.',
 'from',
 'raise',
 'temporary',
 'working',
 'capital',
 'by',
 'have',
 'a',
 'rtc-owned',
 'bank',
 'or',
 'thrift',
 'issue',
 'debt',
 'that',
 'would',
 'not',
 'be',
 'count',
 'on',
 'the',
 'federal',
 'budget',
 '.',
 'the',
 'bill',
 'intend',
 'to',
 'restrict',
 'the',
 'rtc',
 'to',
 'treasury',
 'borrowing',
 'only',
 ',',
 'unless',
 'the',
 'agency',
 'receive',
 'specific',
 'congressional',
 'authorization',
 '.',
 'but',
 'the',
 'rtc',
 'also',
 'require',
 '``',
 'work',
 "''",
 'capital',
 'to',
 'maintain',
 'the',
 'bad',
 'asset',
 'of',
 'thrift',
 'that',
 'be',
 'sell',
 ',',
 'until',
 'the',
 'asset',
 'can',
 'be',
 'sell',
 'separately',
 '.',
 'that',
 'debt',
 'would',
 'be',
 'pay',
 'off',
 'as',
 'the',
 'asset',
 'be',
 'sell',
 ',',
 'leave',


In [45]:
c=[3,4,5]
for i,cc in enumerate(c):
    c[i]=6

print(c)

[6, 6, 6]
